In [1]:
import duckdb

In [2]:
con = duckdb.connect(r"C:\Users\vigne\Desktop\Capstone_old\datasets\Capstone_data_sql.duckdb")

In [3]:
con.execute("show tables").fetchall()

[('allele',),
 ('cross_references',),
 ('gene',),
 ('hgvs4variation',),
 ('organization_summary',),
 ('submission_summary',),
 ('summary_of_conflicting_interpretations',),
 ('var_citations',)]

In [4]:
df = con.execute("select * from allele").fetchdf()

In [5]:
df.rename(columns={"Category": "VariantGeneRelation"}, inplace=True)

In [6]:
df.columns

Index(['AlleleID', 'HgvsNotation', 'GeneID', 'HGNCID', 'ClinicalSignificance',
       'LastEvaluated', 'RS# (dbSNP)', 'RCVAccession', 'PhenotypeIDS',
       'PhenotypeList', 'Origin', 'ChromosomeAccession', 'Chromosome',
       'ReviewStatus', 'NumberSubmitters', 'TestedInGTR', 'OtherIDs',
       'VariationID', 'PositionVCF', 'ReferenceAlleleVCF',
       'AlternateAlleleVCF', 'GenesPerAlleleID', 'VariantGeneRelation', 'MC'],
      dtype='object')

In [7]:
df = df[['AlleleID', 'MC', 'Origin', 'ReferenceAlleleVCF', 
         'AlternateAlleleVCF', 'Chromosome', 'VariantGeneRelation']]


In [8]:
df.head()

,AlleleID,MC,Origin,ReferenceAlleleVCF,AlternateAlleleVCF,Chromosome,VariantGeneRelation
0,15043,SO:0001583|missense_variant,germline,G,A,15,within single gene
1,15044,"SO:0001587|nonsense,SO:0001619|non-coding_tran...",germline,C,T,11,within single gene
2,15045,"SO:0001583|missense_variant,SO:0001619|non-cod...",germline,A,G,11,within single gene
3,15046,"SO:0001583|missense_variant,SO:0001619|non-cod...",germline;paternal,G,A,14,within single gene
4,15048,"SO:0001583|missense_variant,SO:0001627|intron_...",biparental;germline;unknown,G,A,6,within single gene


In [9]:
dfg = con.execute("select * from hgvs4variation").fetchdf()

In [10]:
dfg.drop(["VariationID","NucleotideExpression"],inplace = True, axis = 1)
# dropping dupe alleleID's
dfg = dfg.drop_duplicates(subset=['AlleleID'])
dfg['hgvs_prefix'] = dfg['NucleotideChange'].str[0]
dfg.drop(["NucleotideChange"],inplace = True, axis = 1)

In [11]:
dfg.rename(columns = {'hgvs_prefix':'GenomicLocationData'},inplace = True)

In [12]:
final = df.merge(dfg,
on = "AlleleID",
how='left'
)

In [13]:
final.head()

,AlleleID,MC,Origin,ReferenceAlleleVCF,AlternateAlleleVCF,Chromosome,VariantGeneRelation,GenomicLocationData
0,15043,SO:0001583|missense_variant,germline,G,A,15,within single gene,g
1,15044,"SO:0001587|nonsense,SO:0001619|non-coding_tran...",germline,C,T,11,within single gene,g
2,15045,"SO:0001583|missense_variant,SO:0001619|non-cod...",germline,A,G,11,within single gene,g
3,15046,"SO:0001583|missense_variant,SO:0001619|non-cod...",germline;paternal,G,A,14,within single gene,g
4,15048,"SO:0001583|missense_variant,SO:0001627|intron_...",biparental;germline;unknown,G,A,6,within single gene,g


In [14]:
final.head(10).to_csv('batch_encoder_test.csv', index=False)


In [15]:
con.close()